In [36]:
import pandas as pd
import json
from bokeh.palettes import Category20c
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from bokeh.transform import cumsum, linear_cmap
from bokeh.io import output_file
from bokeh.models import ColumnDataSource, ColorBar
#from bokeh.palettes import RdYlGn as colors
from bokeh.palettes import Plasma256 as colors
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
from math import pi
from bokeh.io import output_notebook
output_notebook() # activate Bokeh output to Jupyter notebook

Loading BokehJS ...

In [37]:
with open("../data/raw_data/test_claims_quantemp.json") as f:
    test_json = json.load(f)

In [199]:
with open("test_evidences_decomposed_custom.json") as f:
    test_json_custom = json.load(f)

In [38]:
len(test_json)

2495

In [200]:
len(test_json_custom)

2495

In [229]:
# For gold standard figures only use this csvs_to_test
# csvs_to_test = {
#     "RoBERTa-MNLI:Gold": "roberta_large_mnli.csv",
#     "BART-MNLI:Gold": "bart_large_mnli.csv",
#     "DeBERTa-NLI:Gold": "deberta-v3-base-tasksource-nli_claimdecomp.csv",
#     "Flan-T5:Gold": "flan-t5-base-eval.csv",
#     "Math-RoBERTa:Gold": "math_roberta.csv",
#     "Numeric-T5:Gold": "numt5.csv",
#     "PASTA:Gold": "pasta-nli_claimdecomp_output.csv",
#     "Elastic-RoBERTa:Gold": "elastic_roberta_claimdecomp_output2.csv",
# }  

# For decomp figures use this csvs_to_test
csvs_to_test = {
    #"GPT-2:Gold": "gpt2_doc.csv",
    "RoBERTa-MNLI:Gold": "roberta_large_mnli.csv",
    "RoBERTa-MNLI:OrigClaim": "roberta_large_mnli_no_decomposition.csv",
    "RoBERTa-MNLI:CLAIMDECOMP": "roberta_large_mnli_flant5.csv",
    "RoBERTa-MNLI:Pseudo Program-FC": "roberta_large_mnli_gpt3.5-turbo.csv",
    "RoBERTa-MNLI:Custom": "roberta_large_mnli_custom.csv",
    "BART-MNLI:Gold": "bart_large_mnli.csv",
    "BART-MNLI:OrigClaim": "bartMNLI_origclaim.csv",
    "BART-MNLI:CLAIMDECOMP": "bartMNLI_decomp_flant5.csv",
    "BART-MNLI:Pseudo Program-FC": "bart_large_mnli_gpt3.5-turbo.csv",
    "BART-MNLI:Custom": "bart_large_mnli_custom.csv",
    "DeBERTa-NLI:Gold": "deberta-v3-base-tasksource-nli_claimdecomp.csv",
    "DeBERTa-NLI:OrigClaim": "deberta_v3_no_decomposition.csv",
    "DeBERTa-NLI:CLAIMDECOMP": "deberta_v3_flant5.csv",
    "DeBERTa-NLI:Pseudo Program-FC": "deberta_v3_gpt3.5-turbo.csv",
    "DeBERTa-NLI:Custom": "deberta_v3_custom.csv",
    "Flan-T5:Gold": "flan-t5-base-eval.csv",
    "Flan-T5:OrigClaim": "flant5_no_decomposition.csv",
    "Flan-T5:CLAIMDECOMP": "flant5_flant5.csv",
    "Flan-T5:Pseudo Program-FC": "flant5_gpt3.5-turbo.csv",
    "Flan-T5:Custom": "flant5_custom.csv",
    
    "Math-RoBERTa:Gold": "math_roberta.csv",
    "Math-RoBERTa:OrigClaim": "math_roberta_no_decomposition.csv",
    "Math-RoBERTa:CLAIMDECOMP": "math_roberta_flant5.csv",
    "Math-RoBERTa:Pseudo Program-FC": "math_roberta_gpt3.5-turbo.csv",
    "Math-RoBERTa:Custom": "math_roberta_custom.csv",
    "Numeric-T5:Gold": "numt5.csv",
    "Numeric-T5:OrigClaim": "numt5_no_decomposition.csv",
    "Numeric-T5:CLAIMDECOMP": "numt5_flant5.csv",
    "Numeric-T5:Pseudo Program-FC": "numt5_gpt3.5-turbo.csv",
    "Numeric-T5:Custom": "numt5_custom.csv",
    "PASTA:Gold": "pasta-nli_claimdecomp_output.csv",
    "PASTA:OrigClaim": "pasta_no_decomposition.csv",
    "PASTA:CLAIMDECOMP": "pasta_flant5.csv",
    "PASTA:Pseudo Program-FC": "pasta_gpt3.5-turbo.csv",
    "PASTA:Custom": "pasta_custom.csv",
    "Elastic-RoBERTa:Gold": "elastic_roberta_claimdecomp_output2.csv",
    "Elastic-RoBERTa:OrigClaim": "elasticRoBERTa_origclaim.csv",
    "Elastic-RoBERTa:CLAIMDECOMP": "elasticRoBERTa_decomp_flant5.csv",
    "Elastic-RoBERTa:Pseudo Program-FC": "elastic_roberta_gpt3.5-turbo.csv",
    "Elastic-RoBERTa:Custom": "elastic_roberta_custom.csv",
}  

# Pies: What the prediction distribution looked like for each model

In [232]:
value_counts = {"Ground Truth": pd.Series([sample['label'] for sample in test_json]).value_counts().sort_index(ascending=False).to_dict()}
for key in csvs_to_test:
    if not csvs_to_test[key]:
        continue
    df = pd.read_csv(csvs_to_test[key])

    
    if key.split(":")[-1] == "Custom" or key == "PASTA:CLAIMDECOMP":
        for tup, test_sample in zip(df.itertuples(), test_json_custom):
            assert tup.claim == test_sample['claim'], (key, tup.claim, test_sample['claim'], ctr)
    else:

        for tup, test_sample in zip(df.itertuples(), test_json):
            assert tup.claim == test_sample['claim'], (key, tup.claim, test_sample['claim'], ctr)

    value_counts[key] = df['verdict'].value_counts().sort_index(ascending=False).to_dict()
value_counts

{'Ground Truth': {'True': 474, 'False': 1423, 'Conflicting': 598},
 'RoBERTa-MNLI:Gold': {'True': 235, 'False': 1814, 'Conflicting': 446},
 'RoBERTa-MNLI:OrigClaim': {'True': 254, 'False': 1547, 'Conflicting': 694},
 'RoBERTa-MNLI:CLAIMDECOMP': {'True': 696, 'False': 1618, 'Conflicting': 181},
 'RoBERTa-MNLI:Pseudo Program-FC': {'True': 1248,
  'False': 1237,
  'Conflicting': 10},
 'RoBERTa-MNLI:Custom': {'True': 209, 'False': 1799, 'Conflicting': 487},
 'BART-MNLI:Gold': {'True': 797, 'False': 1442, 'Conflicting': 256},
 'BART-MNLI:OrigClaim': {'True': 809, 'False': 1181, 'Conflicting': 505},
 'BART-MNLI:CLAIMDECOMP': {'True': 669, 'False': 1287, 'Conflicting': 539},
 'BART-MNLI:Pseudo Program-FC': {'True': 357,
  'False': 1517,
  'Conflicting': 621},
 'BART-MNLI:Custom': {'True': 291, 'False': 1626, 'Conflicting': 578},
 'DeBERTa-NLI:Gold': {'True': 591, 'False': 1454, 'Conflicting': 450},
 'DeBERTa-NLI:OrigClaim': {'True': 465, 'False': 1659, 'Conflicting': 371},
 'DeBERTa-NLI:CLAIM

In [233]:
# Helper function to create a pie chart
def create_pie_chart(data_dict, title):
    data = pd.Series(data_dict).reset_index(name='value').rename(columns={'index': 'label'})
    data['angle'] = data['value']/data['value'].sum() * 2*pi
    data['color'] = Category20c[len(data_dict)]

    # Create a pie chart
    p = figure(height=200, width=400, title=title, toolbar_location="left", tools="hover,save,pan,box_zoom,reset", tooltips="@label: @value", x_range=(-1, 1))
    
    p.wedge(x=0, y=1, radius=0.4, 
            start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'), 
            line_color="white", fill_color='color', legend_field='label', source=data)
    
    p.xgrid.visible = False
    p.ygrid.visible = False
    p.axis.visible = False

    # p.wedge.text(x=0, y=1, angle=cumsum('angle', include_zero=True),
    #              text='label', source=data, text_align='center')

    return p

# # Create pie charts for each of the dataframes
plots = []
for key in value_counts:
    plot = create_pie_chart(value_counts[key], key)
    plots.append(plot)
plots.insert(1, None)
plots.insert(1, None)
plots.insert(3, None)
plots.insert(3, None)

# Arrange the pie charts in a grid layout (2 rows and 4 columns)
grid = gridplot([plots[i:i+5] for i in range(0, len(plots), 5)])

# # Output to an HTML file
# output_file("pie_charts_grid.html")
show(grid)

# Confusion matrices

In [234]:
def create_confusion_matrix(ground_truth, predicted_labels, title):  
    # Compute the confusion matrix
    labels = sorted(set(ground_truth + predicted_labels))  # Ensure labels are consistent across both lists
    conf_matrix = confusion_matrix(ground_truth, predicted_labels, labels=labels)

    conf_matrix_percent = conf_matrix.astype('float')  # Convert to float for division
    row_sums = conf_matrix_percent.sum(axis=1)  # Sum of each row (ground truth label)
    conf_matrix_percent = (conf_matrix_percent.T / row_sums).T * 100  # Normalize and convert to percentage
    
    # Create a DataFrame for easier manipulation
    df_conf_matrix = pd.DataFrame(conf_matrix_percent, index=labels, columns=labels)
    # Create a DataFrame for easier manipulation
    #df_conf_matrix = pd.DataFrame(conf_matrix, index=labels, columns=labels)
    
    # Prepare data for the Bokeh plot
    x_range = labels 
    y_range = labels  # Reverse y-axis for better visualization (top-down)
    values = df_conf_matrix.values.flatten()
    
    # Generate the ColumnDataSource for Bokeh
    source = ColumnDataSource(data=dict(
        x=np.tile(np.arange(len(labels)), len(labels)),
        y=np.repeat(np.arange(len(labels)), len(labels)),
        value=values
    ))
    
    # Set up the color mapping
    mapper = linear_cmap(field_name='value', palette=colors, low=0, high=100)
    
    # Create the figure for plotting
    p = figure(title=title, x_range=x_range, y_range=y_range,
               toolbar_location=None, tools="hover,save,pan,box_zoom,reset", 
               tooltips="@x, @y: @value%", width=300, height=300,
               # x_range=(-0.5, len(labels) - 0.5),  # Add padding to the left and right
               # y_range=(-0.5, len(labels) - 0.5),  # Add padding to the top and bottom
              )
    
    # Create a heatmap
    p.rect(x='x', y='y', width=1, height=1, source=source,
           line_color="white", fill_color=mapper)
    
    # Add color bar
    color_bar = ColorBar(color_mapper=mapper['transform'], width=8, location=(0, 0))
    p.add_layout(color_bar, 'right')
    
    # Customize axis labels
    p.xaxis.axis_label = 'Predicted'
    p.yaxis.axis_label = 'Ground Truth'
    p.xaxis.major_label_orientation = 'vertical'

    return p

In [235]:
ground_truth = [sample['label'] for sample in test_json]
ground_truth_custom = [sample['label'] for sample in test_json_custom]
predicted_labels = {"Ground Truth": ground_truth}
for key in csvs_to_test:
    if not csvs_to_test[key]:
        continue
    df = pd.read_csv(csvs_to_test[key])

    if key.split(":")[-1] == "Custom" or key == "PASTA:CLAIMDECOMP":
        for tup, test_sample in zip(df.itertuples(), test_json_custom):
            assert tup.claim == test_sample['claim']
    else:
        for tup, test_sample in zip(df.itertuples(), test_json):
            assert tup.claim == test_sample['claim']
    
    predicted_labels[key] = df['verdict'].to_list()

In [237]:
# Create confusion matrices for each of the dataframes
plots = []
for key in predicted_labels:
    if key.split(":")[-1] == "Custom" or key == "PASTA:CLAIMDECOMP":
        plot = create_confusion_matrix(ground_truth_custom, predicted_labels[key], key)
    else:
        plot = create_confusion_matrix(ground_truth, predicted_labels[key], key)
    plots.append(plot)
plots.insert(1, None)
plots.insert(1, None)
plots.insert(3, None)
plots.insert(3, None)

# Arrange the pie charts in a grid layout (2 rows and 4 columns)
grid = gridplot([plots[i:i+5] for i in range(0, len(plots), 5)])

# # Output to an HTML file
# output_file("pie_charts_grid.html")
show(grid)

# Qualitative Analysis: Randomly sample some claims for each GOLD model

In [175]:
ground_truth = np.array([sample['label'] for sample in test_json])
predicted_labels = dict()
got_wrong = dict()
for key in csvs_to_test:
    if not csvs_to_test[key]:
        continue
    df = pd.read_csv(csvs_to_test[key])

    for tup, test_sample in zip(df.itertuples(), test_json):
        assert tup.claim == test_sample['claim']

    predicted_labels[key] = df['verdict'].values
    got_wrong[key] = df['verdict'].values != ground_truth
    print(key, sum(got_wrong[key]))

Roberta:Gold 777
BART:Gold 723
Deberta:Gold 692
FlanT5:Gold 814
Math-RoBERTa:Gold 782
Numeric-T5:Gold 875
PASTA:Gold 891
Elastic-RoBERTa:Gold 709


In [176]:
# How many did every model get wrong?
wrong_matrix = np.vstack([got_wrong[key] for key in got_wrong]) # shape #Claims x #Models
trickiness_of_claims = wrong_matrix.sum(axis=0) / len(got_wrong) # percentage of models that got each claim wrong: shape #Claims

In [177]:
trickiness_summary_stats = pd.DataFrame(trickiness_of_claims).describe()
trickiness_summary_stats

,0
count,2495.000000
mean,0.313778
std,0.343281
min,0.000000
25%,0.000000
50%,0.125000
75%,0.625000
max,1.000000


In [149]:
## Get indexes of claims from each quartile

# Get 1st, 2nd, 3rd, and 4th quartile claim indexes
first_quartile = trickiness_of_claims <= trickiness_summary_stats[0]['25%']
second_quartile = np.logical_and(trickiness_of_claims <= trickiness_summary_stats[0]['50%'], trickiness_of_claims > trickiness_summary_stats[0]['25%'])
third_quartile = np.logical_and(trickiness_of_claims > trickiness_summary_stats[0]['50%'], trickiness_of_claims <= trickiness_summary_stats[0]['75%'])
fourth_quartile = trickiness_of_claims > trickiness_summary_stats[0]['75%']

In [150]:
# For each model, get the claims that it got wrong from each quartile
incorrect_claims = {'ALL': {
    '1st': first_quartile, 
    '2nd': second_quartile, 
    '3rd': third_quartile, 
    '4th': fourth_quartile},
}


for key in csvs_to_test:
    if not csvs_to_test[key]:
        continue
    df = pd.read_csv(csvs_to_test[key])

    for tup, test_sample in zip(df.itertuples(), test_json):
        assert tup.claim == test_sample['claim']

    wrong_predicted_labels = df['verdict'].values != ground_truth
    test_claim_indexes = np.array(list(range(len(test_json))))

    incorrect_claims[key] = {
        '1st': test_claim_indexes[np.logical_and(wrong_predicted_labels, first_quartile)], 
        '2nd': test_claim_indexes[np.logical_and(wrong_predicted_labels, second_quartile)], 
        '3rd': test_claim_indexes[np.logical_and(wrong_predicted_labels, third_quartile)], 
        '4th': test_claim_indexes[np.logical_and(wrong_predicted_labels, fourth_quartile)]
    }

    incorrect_claims['ALL'] = {
        '1st': np.logical_and(incorrect_claims['ALL']['1st'], wrong_predicted_labels), 
        '2nd': np.logical_and(incorrect_claims['ALL']['2nd'], wrong_predicted_labels), 
        '3rd': np.logical_and(incorrect_claims['ALL']['3rd'], wrong_predicted_labels), 
        '4th': np.logical_and(incorrect_claims['ALL']['4th'], wrong_predicted_labels)
    }

    print(key, incorrect_claims[key]['1st'].shape, incorrect_claims[key]['2nd'].shape, incorrect_claims[key]['3rd'].shape, incorrect_claims[key]['4th'].shape)


Roberta:Gold (0,) (32,) (352,) (393,)
BART:Gold (0,) (7,) (245,) (471,)
Deberta:Gold (0,) (12,) (229,) (451,)
FlanT5:Gold (0,) (58,) (307,) (449,)
MathRoberta:Gold (0,) (25,) (298,) (459,)
NumT5:Gold (0,) (50,) (375,) (450,)
PASTA:Gold (0,) (46,) (389,) (456,)
Elastic-RoBERTa:Gold (0,) (23,) (269,) (417,)


In [151]:
# Get the number of claims that all models got incorrectly per trickiness level
for key in incorrect_claims['ALL']:
    print(key, incorrect_claims['ALL'][key].sum())

1st 0
2nd 0
3rd 0
4th 156


In [152]:
# For each model, randomly sample a claim it got wrong from each quartile
np.random.seed(42)

sampled_incorrect_claims = dict()
for key in incorrect_claims:
    if key == 'ALL':
        continue
        
    sampled_incorrect_claims[key] = dict()
    for quartile_key in incorrect_claims[key]:
        sampled_incorrect_claims[key][quartile_key] = dict()

        if not incorrect_claims[key][quartile_key].any():
            continue

        random_incorrect_index = np.random.choice(incorrect_claims[key][quartile_key])
        
        sampled_incorrect_claims[key][quartile_key]['index'] = random_incorrect_index
        sampled_incorrect_claims[key][quartile_key]['claim'] = test_json[random_incorrect_index]['claim']
        sampled_incorrect_claims[key][quartile_key]['label'] = test_json[random_incorrect_index]['label']
        sampled_incorrect_claims[key][quartile_key]['predicted'] = predicted_labels[key][random_incorrect_index]
sampled_incorrect_claims['NumT5:Gold']

{'1st': {},
 '2nd': {'index': 1548,
  'claim': 'Wearing two Pnp face masks offers stronger protection than wearing one N95 mask.',
  'label': 'False',
  'predicted': 'True'},
 '3rd': {'index': 2417,
  'claim': 'Says a portfolio managed by the Texas General Land Office earned 22 percent last year while the state’s emergency reserve account experienced a 1 percent gain.',
  'label': 'True',
  'predicted': 'Conflicting'},
 '4th': {'index': 830,
  'claim': '"And while (Ted) Strickland proposed cuts for services for children, he wasted over $250,000 remodeling his bathrooms at the governor’s mansion."',
  'label': 'False',
  'predicted': 'Conflicting'}}

In [81]:
# For each model, randomly sample a claim it got wrong from each quartile
np.random.seed(42)

sampled_incorrect_claims = dict()
for key in incorrect_claims:
    if key == 'ALL':
        continue
        
    sampled_incorrect_claims[key] = dict()
    for quartile_key in incorrect_claims[key]:
        sampled_incorrect_claims[key][quartile_key] = dict()

        if not incorrect_claims[key][quartile_key].any():
            continue

        random_incorrect_index = np.random.choice(incorrect_claims[key][quartile_key])
        
        sampled_incorrect_claims[key][quartile_key]['index'] = random_incorrect_index
        sampled_incorrect_claims[key][quartile_key]['claim'] = test_json[random_incorrect_index]['claim']
        sampled_incorrect_claims[key][quartile_key]['label'] = test_json[random_incorrect_index]['label']
        sampled_incorrect_claims[key][quartile_key]['predicted'] = predicted_labels[key][random_incorrect_index]
sampled_incorrect_claims

{'Roberta:Gold': {'1st': {},
  '2nd': {'index': 1845,
   'claim': '"Every taxpayer owes about $130,000 to pay off the national debt."',
   'label': 'True',
   'predicted': 'Conflicting'},
  '3rd': {'index': 1905,
   'claim': '"My opponent, Rick Gunn, blocked the expansion of Medicaid — costing half a million people health insurance, including at least 23,000 veterans."',
   'label': 'True',
   'predicted': 'Conflicting'},
  '4th': {'index': 2114,
   'claim': 'Says Democratic recall opponent Mahlon Mitchell sent a letter "encouraging folks to boycott more than 100 companies."',
   'label': 'Conflicting',
   'predicted': 'False'}},
 'BART:Gold': {'1st': {},
  '2nd': {'index': 647,
   'claim': '"In New York, when she ran for reelection, she carried 58 of our 62 counties. George Bush had won 40 counties in New York just two years earlier."',
   'label': 'True',
   'predicted': 'Conflicting'},
  '3rd': {'index': 1530,
   'claim': 'Some reports suggest Covid-19 vaccines have caused a 2,000% 

In [153]:
# Randomly sample a claim that every model got wrong
np.random.seed(42)

all_incorrect = np.array(list(range(len(test_json))))[incorrect_claims['ALL']['4th']]

sampled_all_incorrect_index = np.random.choice(all_incorrect)
sampled_all_incorrect_claim = test_json[sampled_all_incorrect_index]['claim']
sampled_all_incorrect_label = test_json[sampled_all_incorrect_index]['label']

model_preds_for_all_incorrect = dict()
for key in predicted_labels:
    model_preds_for_all_incorrect[key] = predicted_labels[key][sampled_all_incorrect_index]

sampled_all_incorrect_index, sampled_all_incorrect_claim, sampled_all_incorrect_label

(1585,
 'In 2017 and 2018, teenage boys widely purchased and used a urethra-covering adhesive band called the Jiftip.',
 'Conflicting')

In [154]:
model_preds_for_all_incorrect

{'Roberta:Gold': 'False',
 'BART:Gold': 'False',
 'Deberta:Gold': 'False',
 'FlanT5:Gold': 'False',
 'MathRoberta:Gold': 'False',
 'NumT5:Gold': 'False',
 'PASTA:Gold': 'False',
 'Elastic-RoBERTa:Gold': 'False'}